In [23]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [25]:
tf.keras.backend.clear_session()

In [26]:
subject = 'Rio Grande Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [27]:
data = pd.read_csv('2003_mo_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH,Rio Grande Do Sul - Consumo de Cimento (t)
0,2003-1,8.192273,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,135.543840,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,0.399576,0.775641,188.093
1,2003-2,8.186359,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,134.468148,2.068579e+08,8.912326e+06,18.211286,1.961183e+08,0.398811,0.775686,170.973
2,2003-3,8.180446,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,134.242963,2.070864e+08,8.917522e+06,18.215621,1.961742e+08,0.397898,0.775731,174.880
3,2003-4,8.174533,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,133.169391,2.073148e+08,8.922718e+06,18.219955,1.962300e+08,0.397042,0.775775,178.111
4,2003-5,8.168619,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,132.046874,2.075433e+08,8.927915e+06,18.224290,1.962858e+08,0.396050,0.775820,184.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.034199,NaN,NaN,NaN,NaN,0.832392,NaN,288.217
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.001080,NaN,NaN,NaN,NaN,0.829742,NaN,272.178
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.052788,NaN,NaN,NaN,NaN,0.826925,NaN,272.665
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.665535,NaN,NaN,NaN,NaN,0.823550,NaN,275.972


In [28]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.966721,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.420132,1.083219,-0.768157,0.374323,0.568222,1.591021,0.807464
188,-0.973649,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.417824,1.070530,-0.760321,0.347418,0.545097,1.635348,0.773102
189,-0.980576,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.381727,1.057841,-0.752485,0.320513,0.521971,1.689944,0.738739
190,-0.987503,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.359803,1.045153,-0.744649,0.293608,0.498846,1.737741,0.704377


In [29]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [30]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.092018,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.647118,1.197586,-0.541459,0.739492,0.925913,0.936850,1.430588
158,-0.148891,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.689363,1.198866,-0.569240,0.731546,0.918346,0.935032,1.419848
159,-0.205765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.746341,1.200147,-0.597022,0.723600,0.910778,0.935266,1.409108
160,-0.262638,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.811207,1.201428,-0.624803,0.715654,0.903211,0.937596,1.398368


In [31]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
157    196.675
158    251.395
159    199.081
160    220.793
161    211.546
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [32]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [33]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [34]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
126,-1.124877,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.966618,0.843865,0.507575,0.979193,1.048843,0.602074,0.845653
127,-1.048798,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.910075,0.861669,0.468267,0.975275,1.050210,0.631855,0.868562
128,-0.972719,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.848166,0.879474,0.428960,0.971356,1.051577,0.662142,0.891471
129,-0.896640,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.792181,0.897278,0.389653,0.967437,1.052943,0.695357,0.914380
130,-0.820561,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.728102,0.915082,0.350345,0.963518,1.054310,0.719249,0.937288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.966721,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.420132,1.083219,-0.768157,0.374323,0.568222,1.591021,0.807464
188,-0.973649,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.417824,1.070530,-0.760321,0.347418,0.545097,1.635348,0.773102
189,-0.980576,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.381727,1.057841,-0.752485,0.320513,0.521971,1.689944,0.738739
190,-0.987503,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.359803,1.045153,-0.744649,0.293608,0.498846,1.737741,0.704377


In [35]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [36]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [37]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [38]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [39]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2172328784, 4222155704, 4273793489, 4150033465, 270734336, 1707932130, 715695781, 1039220334, 3989063490, 2611585916, 807043121, 535985228, 958741132, 2048505108, 1356226302, 201625411, 2197807624, 3154391600, 2589039212, 4191681136, 2283062260, 3816518709, 287802400, 2843383243, 271968777, 1826115079, 742003469, 1541784507, 2030636026, 2389684635, 2399206761, 4145039577, 2844566747, 3853966098, 331636585, 1959202590, 1113936487, 3963291319, 2354949901, 387632666, 4157059226, 238361054, 2650426568, 2735437646, 1102979112, 554419007, 2467554630, 3088167662, 3161157943, 1210383702]


Step: 0 ___________________________________________
val_loss: 135.02882385253906
winner_seed: 2172328784


Step: 1 ___________________________________________
val_loss: 73.30401611328125
winner_seed: 4222155704


Step: 2 ___________________________________________
val_loss: 84.09323120117188


Step: 3 ___________________________________________
val_loss: 66.92819213867188
winner_seed: 4150033465


Step: 4 _

In [40]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 129ms/step - loss: 46193.8047 - val_loss: 1972.7335
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 1622.5234 - val_loss: 1070.5060
Epoch 3/10000
4/4 [==============================] - 0s 12ms/step - loss: 1595.0811 - val_loss: 1208.3932
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 1753.0908 - val_loss: 1070.9121
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 1521.8602 - val_loss: 1185.4655
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 1632.4681 - val_loss: 1268.6406
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 1576.2466 - val_loss: 1055.7097
Epoch 8/10000
4/4 [==============================] - 0s 11ms/step - loss: 1539.4740 - val_loss: 1231.3774
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 1512.1093 - val_loss: 1046.7665
Epoch 10/10000
4/4 [========================

4/4 [==============================] - 0s 12ms/step - loss: 463.7543 - val_loss: 542.2607
Epoch 79/10000
4/4 [==============================] - 0s 9ms/step - loss: 484.2857 - val_loss: 202.4658
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 416.4031 - val_loss: 343.8226
Epoch 81/10000
4/4 [==============================] - 0s 14ms/step - loss: 557.6323 - val_loss: 221.5506
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 377.6917 - val_loss: 281.8533
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 333.5600 - val_loss: 259.1951
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 326.0261 - val_loss: 287.7217
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 412.1712 - val_loss: 321.1774
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 512.7903 - val_loss: 432.6721
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - lo

4/4 [==============================] - 0s 11ms/step - loss: 322.3713 - val_loss: 159.4088
Epoch 156/10000
4/4 [==============================] - 0s 12ms/step - loss: 317.1139 - val_loss: 206.1456
Epoch 157/10000
4/4 [==============================] - 0s 11ms/step - loss: 304.4527 - val_loss: 229.9131
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 305.9679 - val_loss: 183.2656
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 312.2246 - val_loss: 172.4916
Epoch 160/10000
4/4 [==============================] - 0s 13ms/step - loss: 320.2404 - val_loss: 167.7453
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 321.1888 - val_loss: 174.4313
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 316.6335 - val_loss: 244.9790
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 321.4314 - val_loss: 175.6421
Epoch 164/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 10ms/step - loss: 292.6902 - val_loss: 201.9807
Epoch 233/10000
4/4 [==============================] - 0s 9ms/step - loss: 282.2180 - val_loss: 235.9626
Epoch 234/10000
4/4 [==============================] - 0s 10ms/step - loss: 323.8069 - val_loss: 159.4468
Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 270.6080 - val_loss: 185.0447
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 259.1472 - val_loss: 140.1231
Epoch 237/10000
4/4 [==============================] - 0s 10ms/step - loss: 299.3408 - val_loss: 222.3317
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 300.5325 - val_loss: 167.3149
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 241.3308 - val_loss: 523.6207
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 547.9367 - val_loss: 203.5796
Epoch 241/10000
4/4 [==============================] - 0s 11ms/

4/4 [==============================] - 0s 10ms/step - loss: 302.9905 - val_loss: 219.6191
Epoch 310/10000
4/4 [==============================] - 0s 10ms/step - loss: 317.2685 - val_loss: 178.6837
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 350.3914 - val_loss: 166.1693
Epoch 312/10000
4/4 [==============================] - 0s 10ms/step - loss: 333.3057 - val_loss: 179.2222
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 300.1385 - val_loss: 315.1234
Epoch 314/10000
4/4 [==============================] - 0s 10ms/step - loss: 424.0506 - val_loss: 194.1833
Epoch 315/10000
4/4 [==============================] - 0s 10ms/step - loss: 302.5378 - val_loss: 210.6738
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 353.3591 - val_loss: 198.9683
Epoch 317/10000
4/4 [==============================] - 0s 11ms/step - loss: 297.3761 - val_loss: 177.1631
Epoch 318/10000
4/4 [==============================] - 0s 12ms

...

Epoch 1615/10000
4/4 [==============================] - 0s 12ms/step - loss: 114.0265 - val_loss: 155.8683
Epoch 1616/10000
4/4 [==============================] - 0s 10ms/step - loss: 107.9266 - val_loss: 175.4044
Epoch 1617/10000
4/4 [==============================] - 0s 9ms/step - loss: 117.5422 - val_loss: 149.3176
Epoch 1618/10000
4/4 [==============================] - 0s 9ms/step - loss: 146.9053 - val_loss: 149.7413
Epoch 1619/10000
4/4 [==============================] - 0s 10ms/step - loss: 118.9461 - val_loss: 148.4279
Epoch 1620/10000
4/4 [==============================] - 0s 9ms/step - loss: 131.5759 - val_loss: 254.1889
Epoch 1621/10000
4/4 [==============================] - 0s 10ms/step - loss: 143.5598 - val_loss: 209.5756
Epoch 1622/10000
4/4 [==============================] - 0s 9ms/step - loss: 131.6607 - val_loss: 238.8045
Epoch 1623/10000
4/4 [==============================] - 0s 10ms/step - loss: 146.8743 - val_loss: 162.3746
Epoch 1624/10000
4/4 [===================

Epoch 1692/10000
4/4 [==============================] - 0s 10ms/step - loss: 125.4660 - val_loss: 183.7199
Epoch 1693/10000
4/4 [==============================] - 0s 10ms/step - loss: 121.9841 - val_loss: 159.5671
Epoch 1694/10000
4/4 [==============================] - 0s 12ms/step - loss: 99.1552 - val_loss: 181.3203
Epoch 1695/10000
4/4 [==============================] - 0s 11ms/step - loss: 113.9282 - val_loss: 163.8855
Epoch 1696/10000
4/4 [==============================] - 0s 10ms/step - loss: 108.1782 - val_loss: 178.9321
Epoch 1697/10000
4/4 [==============================] - 0s 11ms/step - loss: 125.4197 - val_loss: 230.6169
Epoch 1698/10000
4/4 [==============================] - 0s 11ms/step - loss: 127.2306 - val_loss: 180.6942
Epoch 1699/10000
4/4 [==============================] - 0s 10ms/step - loss: 97.4382 - val_loss: 203.7271
Epoch 1700/10000
4/4 [==============================] - 0s 10ms/step - loss: 108.7154 - val_loss: 216.9071
Epoch 1701/10000
4/4 [=================

4/4 [==============================] - 0s 10ms/step - loss: 129.0368 - val_loss: 205.4281
Epoch 1769/10000
4/4 [==============================] - 0s 11ms/step - loss: 90.2243 - val_loss: 214.3054
Epoch 1770/10000
4/4 [==============================] - 0s 11ms/step - loss: 110.8162 - val_loss: 193.8176
Epoch 1771/10000
4/4 [==============================] - 0s 11ms/step - loss: 99.3068 - val_loss: 166.2278
Epoch 1772/10000
4/4 [==============================] - 0s 11ms/step - loss: 94.9680 - val_loss: 255.0931
Epoch 1773/10000
4/4 [==============================] - 0s 11ms/step - loss: 119.1108 - val_loss: 153.4936
Epoch 1774/10000
4/4 [==============================] - 0s 12ms/step - loss: 128.3115 - val_loss: 299.4404
Epoch 1775/10000
4/4 [==============================] - 0s 12ms/step - loss: 137.0790 - val_loss: 189.1260
Epoch 1776/10000
4/4 [==============================] - 0s 11ms/step - loss: 102.8779 - val_loss: 167.7570
Epoch 1777/10000
4/4 [==============================] - 0

Epoch 1845/10000
4/4 [==============================] - 0s 12ms/step - loss: 122.8676 - val_loss: 196.4181
Epoch 1846/10000
4/4 [==============================] - 0s 12ms/step - loss: 116.5487 - val_loss: 193.4811
Epoch 1847/10000
4/4 [==============================] - 0s 12ms/step - loss: 124.3253 - val_loss: 237.3473
Epoch 1848/10000
4/4 [==============================] - 0s 11ms/step - loss: 175.9872 - val_loss: 202.7885
Epoch 1849/10000
4/4 [==============================] - 0s 12ms/step - loss: 119.1623 - val_loss: 211.3015
Epoch 1850/10000
4/4 [==============================] - 0s 12ms/step - loss: 137.2863 - val_loss: 148.2121
Epoch 1851/10000
4/4 [==============================] - 0s 11ms/step - loss: 173.1167 - val_loss: 209.2029
Epoch 1852/10000
4/4 [==============================] - 0s 12ms/step - loss: 122.8879 - val_loss: 226.4789
Epoch 1853/10000
4/4 [==============================] - 0s 11ms/step - loss: 151.8983 - val_loss: 188.8098
Epoch 1854/10000
4/4 [===============

4/4 [==============================] - 0s 10ms/step - loss: 260.7951 - val_loss: 285.7158
Epoch 1922/10000
4/4 [==============================] - 0s 10ms/step - loss: 256.5353 - val_loss: 172.0130
Epoch 1923/10000
4/4 [==============================] - 0s 10ms/step - loss: 204.9881 - val_loss: 199.3268
Epoch 1924/10000
4/4 [==============================] - 0s 11ms/step - loss: 200.8127 - val_loss: 144.5981
Epoch 1925/10000
4/4 [==============================] - 0s 10ms/step - loss: 187.6094 - val_loss: 174.6497
Epoch 1926/10000
4/4 [==============================] - 0s 10ms/step - loss: 225.3884 - val_loss: 185.8731
Epoch 1927/10000
4/4 [==============================] - 0s 10ms/step - loss: 189.7379 - val_loss: 169.7458
Epoch 1928/10000
4/4 [==============================] - 0s 11ms/step - loss: 188.3092 - val_loss: 162.2015
Epoch 1929/10000
4/4 [==============================] - 0s 10ms/step - loss: 190.2141 - val_loss: 147.0456
Epoch 1930/10000
4/4 [==============================] 

In [41]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [42]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,216.150055,216.531326,216.835129,216.723099,216.431488,216.045883,214.808212,214.038177,213.760483,213.953934,213.25209,212.871719,212.405533,212.060333,212.268082,211.853897,211.607895,211.439468,210.324722,210.370026,208.796036,207.354233,206.799911,206.030396,204.390457,206.247452,203.101608,209.950272,208.578964,208.139725
Target,244.68,252.183,249.403,208.299,238.414,213.179,221.534,212.139,228.994,226.827,186.855,253.021,194.988,244.406,201.372,251.369,242.782,201.288,226.547,228.5,223.896,234.132,207.586,228.583,234.565,249.519,230.086,233.136,233.8,223.214
Error,28.529938,35.651672,32.567871,8.424103,21.982513,2.866882,6.725784,1.89917,15.233521,12.873062,26.397095,40.149277,17.417526,32.345673,10.896088,39.515106,31.174103,10.151474,16.222275,18.129974,15.09996,26.777771,0.786087,22.552597,30.174545,43.271545,26.98439,23.18573,25.221039,15.07428


In [43]:
display(mae)
display(mape)

21.276035

0.0912805

In [44]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [45]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined